In [81]:
import numpy as np
import csv
import random
import copy
import warnings
warnings.filterwarnings("ignore")

In [82]:
class gene():
    def __init__(self,name, data):
        self.name = name
        self.data = data
    def __repr__(self):
        return f"point:{self.name}"
    
    def __eq__(self, other):
        return self.name == other.name

In [83]:
class Chromosome():
    def __init__(self, list_genes):
        self.fitness = 0 
        self.cost = 0 
        self.list_genes = list_genes 
    def __repr__(self):
        return f'{self.list_genes} cost: {self.cost} fitness:{self.fitness}'

In [84]:
def create_data(path):
    lst = []
    with open(path, 'r') as f:
            lines = list(csv.DictReader(f))
    for line in lines:
            lst.append(gene(line['City'], [line['x'], line ['y']]))
    return lst

In [85]:
def distance(povit, point):
    povit = np.array(povit, dtype= float)
    point = np.array(point, dtype= float)
    dist =np.sum((point - povit)**2)**0.5
    return dist

In [86]:
def Matrix_distance(intinal_chromosome):
    distance_matrix = np.zeros((len(intinal_chromosome), len(intinal_chromosome)))
    for i in range(len(intinal_chromosome)):
        for j in range(len(intinal_chromosome)):
            if i != j:
                distance_matrix[i,j] = distance(intinal_chromosome[i].data,intinal_chromosome[j].data)
    return distance_matrix

In [87]:
def cost_chromosome(chromosome, distance_matrix):
    total_distance = 0
    for i in range(len(chromosome.list_genes)-1):
        city1 = chromosome.list_genes[i]
        city2 = chromosome.list_genes[i+1]
        total_distance += distance_matrix[int(city1.name)-1, int(city2.name)-1]
    return total_distance +distance(chromosome.list_genes[-1].data,chromosome.list_genes[0].data)

In [89]:
def create_population(num,distance_matrix):
    lst = create_data('15-Points.csv')
    population = []
    for i in range(num):
        c = Chromosome(list(np.random.permutation(lst)))
        # return to the start city again
        c.cost = cost_chromosome(c, distance_matrix)
        c.fitness = 1/c.cost 
        population.append(c)

    return population

In [90]:
def compersion(solution1, solution2):
    if solution1.cost < solution2.cost:
        return solution1
    else:
        return solution2


In [91]:
def formon_matrix(oldformon, pop, lambd):
    new_formon = copy.deepcopy(oldformon) * (1-lambd)
    for Chromosome in pop:
        ph_val = 1 / Chromosome.cost
        for i in range(len(Chromosome.list_genes)-1):
                c1 = int(Chromosome.list_genes[i].name) - 1
                c2 = int(Chromosome.list_genes[i+1].name)- 1

                new_formon[c1, c2] += ph_val        
                new_formon[c2, c1] += ph_val

    return new_formon

In [92]:
def construction_solution(lst,new_formon,distance_matrix, alpha, beta, size):
      new_gene =[]
      while True:
            k= random.choices(range(len(lst)))[0]
            k = lst[k]
            path = []
            path.append(k)
            unvisited_cities = [gene for gene in lst]
            
            
            for j in range(len(lst)-1):
                  probabilities=[]
                  unvisited_cities.remove(k)
                  for i in range(len(lst)):
                        if  lst[i] in path:
                              #print('control', i)
                              continue
                        probability = (((new_formon[int(k.name)-1,i])**alpha) * ((1/distance_matrix[int(k.name)-1,i])**beta))
                        probabilities.append(probability)
                      
                  probabilities = np.array(probabilities) / np.sum(probabilities)
                  #print(path)
                  #print(dict_pop)
                  
                  #print(prop)
                  next_city  = np.random.choice(list(unvisited_cities), p=probabilities)
            
                  path.append(next_city)
                  k = next_city
            ch = Chromosome(path)
            new_gene.append(ch)
            if len(new_gene) == size:
                  return new_gene

In [93]:
def sort_fitnesses(population):
    population =sorted(population,key =lambda x:x.fitness, reverse= True)
    return population

In [94]:
def prepare_pop(population,distance_matrix):
    new_gene = [] 
    for pop in population:
        pop.cost = cost_chromosome(pop, distance_matrix)
        pop.fitness = 1/pop.cost
        #pop.list_genes.append(pop.list_genes[0])
        new_gene.append(pop)

    return sort_fitnesses(new_gene)

In [95]:
if __name__ == "__main__":

    size_pop = 100
    alpha = 1
    beta = 2
    num_gene = 10
    oldformon = np.zeros((15,15))+0.000000000001
    lambd = 0.7

    lst = create_data('15-Points.csv')
    distance_matrix = Matrix_distance(lst)
    population = create_population(size_pop,distance_matrix)
    new_formon = formon_matrix(oldformon, population,lambd)
    #print(new_formon)
    best_one_for_all_generation=[]

    for i in range (num_gene):   
              population = construction_solution(lst,new_formon,distance_matrix, alpha, beta, size_pop)
              population = prepare_pop(population,distance_matrix)
              best_one_for_all_generation.append(population[0])
              new_formon = formon_matrix(new_formon, population, lambd)

    result = sort_fitnesses(best_one_for_all_generation)[0]
    result.list_genes.append(result.list_genes[0])
    print(result)

[point:5, point:7, point:3, point:12, point:14, point:10, point:8, point:6, point:4, point:11, point:1, point:13, point:2, point:15, point:9, point:5] cost: 284.3810904080331 fitness:0.0035164082062038265


In [96]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt

In [97]:
def plot_sol(solution):
    x = []
    y = []
    names=[]
    for gene in solution.list_genes:
        x.append(float(gene.data[0]))
        y.append(float(gene.data[1]))
        names.append('point'+gene.name)
    fig, ax = plt.subplots()
    def animate(i):
        ax.clear()
        ax.plot(x[:i+1], y[:i+1], marker = 'o')
    plt.plot(x, y, marker = 'o')
    for i in range(len(x)):
        plt.text(x[i]+0.1, y[i]+0.1, names[i], fontsize=10)
    plt.title(f'Ant Colony algorithm Cost {result.cost}')
    plt.show()

In [98]:
plot_sol(result)

In [99]:
x = []
y = []
names = []
for gene in result.list_genes:
        x.append(float(gene.data[0]))
        y.append(float(gene.data[1]))
        names.append('point'+gene.name)

fig, ax = plt.subplots()

def animate(i):
    ax.clear()
    ax.plot(x[:i+1], y[:i+1], marker = 'o')
    plt.text(x[i]+0.1, y[i]+0.1, names[i], fontsize=10)
    plt.title(f'Ant Colony algorithm Cost {result.cost}')

    plt.ylim(-50, 30)
    plt.xlim(-90, 10)

ani = FuncAnimation(fig, animate, frames=20, interval=300, repeat=False)
plt.show()